# Universidad del Valle de Guatemala
# (CC3094) Security Data Science
# Laboratorio 1 - Detección de Phishing

Miembros del equipo de trabajo:
- Yongbum Park (20117)
- Santiago Taracena Puga (20017)

## Introducción

En el ámbito actual de la ciberseguridad, la detección y prevención de ataques de phishing se han convertido en prioridades esenciales para salvaguardar la integridad de la información y la privacidad de los usuarios en línea. El phishing, basado en la astuta manipulación psicológica de las víctimas, se ha erigido como una de las tácticas más eficaces utilizadas por ciberdelincuentes para obtener información confidencial, como credenciales de acceso a cuentas bancarias, datos personales y contraseñas.

El presente informe aborda la implementación de un modelo de Machine Learning (ML) destinado a clasificar URLs como legítimas o phishing, fundamentándose en el análisis de características intrínsecas de las direcciones web. Este enfoque busca contrarrestar los sofisticados métodos empleados por los atacantes, quienes, a través de ingeniería social, logran crear réplicas convincentes de sitios web legítimos para engañar a los usuarios.

El laboratorio se estructura en dos partes esenciales: la primera se centra en la ingeniería de características, exploración de datos, derivación de características y preprocesamiento, mientras que la segunda se enfoca en la implementación de modelos de ML y la evaluación de su desempeño. Todo el proceso se desarrollará en el lenguaje de programación Python, aprovechando herramientas y librerías especializadas.

La detección de phishing no solo constituye un desafío técnico, sino también un imperativo para mitigar riesgos financieros y salvaguardar la confidencialidad de la información sensible. Este laboratorio proporcionará una perspectiva práctica sobre cómo abordar esta problemática mediante la aplicación de técnicas de ML, ofreciendo a los participantes una valiosa experiencia en la intersección de la seguridad informática y la ciencia de datos.

## Parte 1 - Ingeniería de Características

En la primera fase de este laboratorio, nos enfocaremos en la Ingeniería de Características, un paso crucial para dotar al modelo de Machine Learning de la capacidad de discernir entre URLs legítimas y potenciales intentos de phishing. Comenzaremos por cargar el conjunto de datos proporcionado en un dataframe de pandas, permitiendo una visualización inicial de cinco observaciones para comprender la estructura del mismo. A continuación, evaluaremos la distribución de las etiquetas en la columna 'status', identificando la proporción de URLs etiquetadas como 'legit' y 'phishing', con el objetivo de determinar si el conjunto de datos presenta un equilibrio adecuado.

Posteriormente, nos sumergiremos en la Derivación de Características, donde exploraremos las ventajas del análisis de una URL en comparación con otros datos, como el tiempo de vida del dominio o las características de la página web. Inspirándonos en artículos especializados en la clasificación de phishing, identificaremos al menos quince funciones basadas en estas investigaciones para enriquecer nuestro conjunto de datos con características relevantes. El proceso de preprocesamiento será esencial para convertir la variable categórica 'status' en una variable binaria, eliminar la columna del dominio y realizar ajustes necesarios para garantizar la coherencia y eficacia del análisis. Finalmente, emplearemos la herramienta pandas_profiling para generar un reporte de perfil que nos proporcionará insights cruciales sobre las características del conjunto de datos y nos guiará en la selección de las características más significativas para la detección de phishing.

### Exploración de datos

Lo primero que debemos realizar, como en todo procedimiento de Ingeniería de Características y procedimientos relacionados a Ingeniería de Datos antes de comenzar con el entrenamiento de un nuevo modelo de aprendizaje de máquina, es familiarizarnos con los datos que nos han sido proporcionados. Antes de cualquier otra cosa, necesitamos hacer uso de la librería de pandas para poder leer el dataset que nos ha sido proporcionado.

In [34]:
!pip install pydantic==1.8.2
!pip install --upgrade pandas-profiling


Defaulting to user installation because normal site-packages is not writeable
  Using cached pydantic-1.8.2-cp39-cp39-win_amd64.whl (1.9 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.0
    Uninstalling pydantic-2.6.0:
      Successfully uninstalled pydantic-2.6.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.6.4 requires pydantic>=2, but you have pydantic 1.8.2 which is incompatible.
anaconda-cloud-auth 0.1.4 requires semver<3, but you have semver 3.0.2 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
  Using cached pydantic-2.6.0-py3-none-any.whl (394 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.8.2
    Uninstalling pydantic-1.8.2:
      Successfully uninstalled pydantic-1.8.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.6.0 which is incompatible.
anaconda-cloud-auth 0.1.4 requires semver<3, but you have semver 3.0.2 which is incompatible.


In [35]:
# Instrucción para importar la librería pandas.
import pandas as pd

Con pandas a nuestra disposición, podemos proceder a leer el dataset que tenemos qué utilizar para el laboratorio. Esto lo podemos realizar utilizando la función `read_csv`, que nos permite leer un archivo con extensión .csv y convertirlo a un DataFrame de pandas, mucho más fácil de manipular.

In [36]:
# Proceso para leer el dataset y colocarlo en un DataFrame.
dataset = pd.read_csv("./data/dataset_phishing.csv")

Algo que se solicita en las instrucciones del laboratorio es poder realizar una observación rápida de un ejemplo de cinco entradas del dataset. Esto lo podemos realizar utilizando la función `head`, que nos retorna las primeras cinco entradas de un DataFrame de pandas.

In [37]:
# Primeras cinco filas del dataset.
dataset.head()

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


Posteriormente necesitamos verificar si el dataset se encuentra balanceado, ya que esta característica es particularmente útil al momento de entrenar un modelo que sea capaz de clasificar entre dos, valga la redundancia, clases diferentes. Esto lo podemos observar, en primer lugar, averiguando cuántas filas tiene nuestro dataset. Esto se puede realizar observando la propiedad `shape` presente en todos los DataFrames de pandas.

In [38]:
# Tupla con las filas y columnas del dataset.
dataset.shape

(11430, 2)

Sabiendo que contamos con 11,430 filas en el dataset, podemos observar cuántas de estas filas corresponden a entradas legítimas o entradas de phishing. A continuación se realiza el query para observar cuántas filas hay de cada una de las clases.

In [39]:
# Cantidad de entradas legítimas del dataset.
dataset[dataset["status"] == "legitimate"].shape[0]

5715

In [40]:
# Cantidad de entradas de phishing del dataset.
dataset[dataset["status"] == "phishing"].shape[0]

5715

Podemos notar cómo el dataset se encuentra perfectamente balanceado, ya que tenemos 5,715 entradas de cada clase para hacer de cada una el 50% del dataset. Al tener esta característica, nos evitamos la gran mayoría de procedimientos manuales para balancear el dataset antes de entrenar al modelo.

# Largo de URL

In [41]:
from codes.url_lenght import url_lenght
legit = dataset[dataset["status"] == "legitimate"]
df = legit["url"].tolist()
urls_instance = url_lenght(df)
urls_instance.print_urls()



Estadísticas generales:
Mínimo: 12 caracteres
Máximo: 557 caracteres
Promedio: 47.38 caracteres


In [42]:
from codes.url_lenght import url_lenght
phishing = dataset[dataset["status"] == "phishing"]
df = phishing["url"].tolist()
urls_instance = url_lenght(df)
urls_instance.print_urls()


Estadísticas generales:
Mínimo: 15 caracteres
Máximo: 1641 caracteres
Promedio: 74.86 caracteres


# Largo de Pathname

In [43]:
from codes.pathname_lenght import pathname_lenght
legit = dataset[dataset["status"] == "legitimate"]
df = legit["url"].tolist()
path_instance = pathname_lenght(df)
path_instance.print_paths()


['/router.php', '/tracks/gateway-motorsports-park/', '/425836/joshwigler/the-amazing-race-host-phil-keoghan-previews-the-season-27-premiere/', '/Astro_MemoNew/Profilo.asp', '/tcp-port-21-818146', '/top-best-mp3-downloader-app-for-android-free-music-download/', '/en/games/nintendo-switch/nintendo-switch/', '/marketing/email-open-click-rate-benchmark', '/games/port-and-starboard', '/computer-tips/how-to-change-mac-address/', '/question/index', '/cms', '/wiki/Databases_and_data_access_APIs', '/dictionary/tabloid', '/oes/current/oes292035.htm', '/wiki/MultiMedia_Card', '/tip/Comparisons-of-coaxial-cable-fiber-optic-cable-and-twisted-pair-cable', '/our-wines', '/g/s/id/pdf_pub/faq/faq002478/en/Scan_To_FTP.pdf', '/examples/shortest-path-problem.html', '/statistics/dictionary.aspx', '/free-icons/parking_403', '/FreeDownloads/RouterTable.pdf', '/wiki/NIC_Bank', '/topics/american-civil-war/gettysburg-address', '/mac-address/', '/wiki/Gateway', '/mimosagrillclt', '/news/local/thief-breaks-into-s

In [44]:
from codes.pathname_lenght import pathname_lenght
phishing = dataset[dataset["status"] == "phishing"]
df = phishing["url"].tolist()
path_instance = pathname_lenght(df)
path_instance.print_paths()


['/V4/validation/a111aedc8ae390eabcfa130e041a10a4', '/ap/89e6a3b4b063b8d/', '/V4/validation/ba4b8bddd7958ecb8772c836c2969531', '/l7ceeid6.html', '/time3/', '/postamok/d39a2/source', '/cs/', '/deliver/D2017HL/u.php', '/ap/bb14d7ff1fcbf29', '/ap/88b142e778d73f7', '/rechnung-376440790464490488&amp', '/Apollo/bc28a52c2070ca93176244f59e1f19c6', '/home/components/com_user/bbtonline.html', '/jpg/www.global.visa.com/myca/oce/emea/action/request_type=un_Activation/visa.html', '/wp-content/plugins/portfolio-cpt/tax/login.php', '/amzn', '/wp-content/uploads/2019/07/tpg/9ac6ba30463d0fb7974c06843717284c/', '/:b:/g/personal/accounts_grassform_co_uk/EYsmX_DhwSVAlooW1ETKf50BIChCl1rrAfmqvTLvySTLQQ', '/HotmailNEW/', '/catalog/language/english/product/special/index.php', '/form-4850478/', '/mot', '/Hao/YouTube/ba.htm', '/application/data/', '/aa/test.php', '/V4/validation/44ccd8b487c7fe797dc775e919236002', '/site/ilyasdxih', '/forms/d/e/1FAIpQLSedi778KA_QpFW-6VseK02w_qghyAY4KrmhVAHDl-qBzUQytg/viewform', 

# Largo del domain

In [45]:
from codes.domain_lenght import domain_lenght
legit = dataset[dataset["status"] == "legitimate"]
df = legit["url"].tolist()
domain_instance = domain_lenght(df)
domain_instance.print_paths()

['www.crestonwood.com', 'rgipt.ac.in', 'www.iracing.com', 'www.mutuo.it', 'vamoaestudiarmedicina.blogspot.com', 'parade.com', 'www.astrologyonline.eu', 'www.lifewire.com', 'technofizi.net', 'www.missfiga.com', 'www.chiefarchitect.com', 'www.2345daohang.com', 'www.game.co.uk', 'blog.hubspot.com', 'sophie-world.com', 'www.provenancevineyards.com', 'www.online-tech-tips.com', 'www.enkiquotes.com', 'answers.yahoo.com', 'www.chiostrodelbramante.it', 'yummy-cummy-in-my-tummy.tumblr.com', 'www.j-net.cn', 'press-preview.weebly.com', 'taccs.hu', 'wiki.openstreetmap.org', 'www.merriam-webster.com', 'www.bls.gov', 'ru.wikipedia.org', 'searchnetworking.techtarget.com', 'www.provenancevineyards.com', 'support.brother.com', 'www.excel-easy.com', 'stattrek.com', 'www.flaticon.com', 'www.bobsplans.com', 'en.wikipedia.org', 'www.history.com', 'remontka.pro', 'en.wikipedia.org', 'twitter.com', 'www.calculate.co.il', 'www.whio.com', 'www.markspcsolution.com', 'www.mummyandmini.com', 'www.showplaceicon.co

In [46]:
from codes.domain_lenght import domain_lenght
phishing = dataset[dataset["status"] == "phishing"]
df = phishing["url"].tolist()
domain_instance = domain_lenght(df)
domain_instance.print_paths()

['shadetreetechnology.com', 'support-appleld.com.secureupdate.duilawyeryork.com', 'appleid.apple.com-app.es', 'www.shadetreetechnology.com', 'html.house', 'wave.progressfilm.co.uk', 'beta.kenaidanceta.com', 'www.ktplasmachinery.com', 'batvrms.net', 'support-appleld.com.secureupdate.duilawyeryork.com', 'polarklimatsgserver.blogspot.com', 'secureupdate.appleld.com.duilawyeryork.com', 'www.inbioma.pe', 'mno.naomigoff.com', 'xh167894743.el.r.appspot.com', 'dichvuvnpt.com', 'swallowthisbitchpics.com', 'realestateinvestordatabase.com', 'vufd.hyperphp.com', 'www.courgeon-immobilier.fr', 'grassform-my.sharepoint.com', 'hotmailsecure.brilliantbokeh.com', 'vaportotal.net', 'www.123formbuilder.com', 'sura.careervidi.com', 'phan.thuchao.free.fr', 'www.astroresidential.co.uk', 'www.nakamistrad.com', 'www.shadetreetechnology.com', 'sites.google.com', 'docs.google.com', 'sharedpont77wwjxjx.df.r.appspot.com', 'mailserver67187.uc.r.appspot.com', 'www.storybookalpacas.com', 'storage.cloud.google.com', '

# Largo del filename

In [47]:
from codes.filename_lenght import filename_lenght
legit = dataset[dataset["status"] == "legitimate"]
df = legit["url"].tolist()
filename_instance = filename_lenght(df)
filename_instance.print_paths()

['router.php', 'Profilo.asp', 'oes292035.htm', 'Scan_To_FTP.pdf', 'shortest-path-problem.html', 'dictionary.aspx', 'RouterTable.pdf', 'router-password-kracker.php', '5-best-free-network-packet-sniffer.html', 'mms.html', 'enabling-google-chrome-menu-bar-78783.html', 'cell2_activetran.html', 'registration.aspx', 'resolution.htm', 'nat.pdf', 'cat5-5e-6-standards.html', 'warp-speed2.htm', 'Sports-Hub-Traffic-Brochure.pdf', 'R42078.pdf', 'line-segment.html', 'paraport.htm', 'com.smartprojects.ramoptimization', 'index.php', 'Search-Engines-Future-User-Preferences-Artificial-Intelligence.html', 'azadet_1_0799.pdf', 'article.html', 'jsref_obj_date.asp', 'electronic-keyboard.html', 'index.php', 'bittorrent1.htm', 'tabloid-paper-dimensions.html', 'mnhistory.cgi', '210625-bluetooth-peripheral-device-driver-device-unplugged-not-found.html', 'DNA-meaning-definition.asp', 'rfc2812.txt', 'osi_model.html', 'collision-report.aspx', 'layout-piano-keys.html', '111893.aspx', '319-resolving-dns-http-redire

In [48]:
from codes.filename_lenght import filename_lenght
phishing = dataset[dataset["status"] == "phishing"]
df = phishing["url"].tolist()
filename_instance = filename_lenght(df)
filename_instance.print_paths()

['l7ceeid6.html', 'u.php', 'bbtonline.html', 'visa.html', 'login.php', 'index.php', 'ba.htm', 'test.php', 'xp.php', 'index2.html', 'index.htm', 'com.htm', 'index.css', 'ordem.html', 'index.php', 'ifram2.php', '928375602332311ochttp3A2F2F2Fws2FISAPIdllFM2MContact26item3B14276952031324requestePPd3DwolandgioielliPPP26qidQ-jlc@amahousse.com.html', 'index.php', 'IDCHECK.html', 'sdnconconsadf.html', 'produto.php', 'login.alibaba.com.php', 'sitekeyverification.php', 'cad.html', 'order.php', 'index.html', 'cmcst-wl.html', 'Login-Acc.php', 'index.php', 'habbo-crediti-gratis-sicuro-100.html', 'index.html', 'r.htm', 'sbcglobal.net.htm', 're.html', 'Hosted.aspx', 'UPS.htm', 'u.php', 'xp.php', 'erreur.htm', 'login.php', 'attractionvk.html', 'activation.php', 'username.php', 'Login.php', 'r1.php', 'login.php', 'index2.php', 'webmail.php', 'webmail.php', 'home.html', 'index.php', 'webmail.php', 'index.php', 'login.php', 'ScotiaWeb2.php', 'index.php', 'off365.html', 'index759b.html', 'index.php', 'ind

# Numero de vocales en el domain

In [49]:
from codes.num_vocals_domain import num_vocals_domain
legit = dataset[dataset["status"] == "legitimate"]
df = legit["url"].tolist()
vocal = num_vocals_domain(df)
vocal.print_num()

['www.crestonwood.com', 'rgipt.ac.in', 'www.iracing.com', 'www.mutuo.it', 'vamoaestudiarmedicina.blogspot.com', 'parade.com', 'www.astrologyonline.eu', 'www.lifewire.com', 'technofizi.net', 'www.missfiga.com', 'www.chiefarchitect.com', 'www.2345daohang.com', 'www.game.co.uk', 'blog.hubspot.com', 'sophie-world.com', 'www.provenancevineyards.com', 'www.online-tech-tips.com', 'www.enkiquotes.com', 'answers.yahoo.com', 'www.chiostrodelbramante.it', 'yummy-cummy-in-my-tummy.tumblr.com', 'www.j-net.cn', 'press-preview.weebly.com', 'taccs.hu', 'wiki.openstreetmap.org', 'www.merriam-webster.com', 'www.bls.gov', 'ru.wikipedia.org', 'searchnetworking.techtarget.com', 'www.provenancevineyards.com', 'support.brother.com', 'www.excel-easy.com', 'stattrek.com', 'www.flaticon.com', 'www.bobsplans.com', 'en.wikipedia.org', 'www.history.com', 'remontka.pro', 'en.wikipedia.org', 'twitter.com', 'www.calculate.co.il', 'www.whio.com', 'www.markspcsolution.com', 'www.mummyandmini.com', 'www.showplaceicon.co

In [50]:
from codes.num_vocals_domain import num_vocals_domain
phishing = dataset[dataset["status"] == "phishing"]
df = phishing["url"].tolist()
vocal = num_vocals_domain(df)
vocal.print_num()

['shadetreetechnology.com', 'support-appleld.com.secureupdate.duilawyeryork.com', 'appleid.apple.com-app.es', 'www.shadetreetechnology.com', 'html.house', 'wave.progressfilm.co.uk', 'beta.kenaidanceta.com', 'www.ktplasmachinery.com', 'batvrms.net', 'support-appleld.com.secureupdate.duilawyeryork.com', 'polarklimatsgserver.blogspot.com', 'secureupdate.appleld.com.duilawyeryork.com', 'www.inbioma.pe', 'mno.naomigoff.com', 'xh167894743.el.r.appspot.com', 'dichvuvnpt.com', 'swallowthisbitchpics.com', 'realestateinvestordatabase.com', 'vufd.hyperphp.com', 'www.courgeon-immobilier.fr', 'grassform-my.sharepoint.com', 'hotmailsecure.brilliantbokeh.com', 'vaportotal.net', 'www.123formbuilder.com', 'sura.careervidi.com', 'phan.thuchao.free.fr', 'www.astroresidential.co.uk', 'www.nakamistrad.com', 'www.shadetreetechnology.com', 'sites.google.com', 'docs.google.com', 'sharedpont77wwjxjx.df.r.appspot.com', 'mailserver67187.uc.r.appspot.com', 'www.storybookalpacas.com', 'storage.cloud.google.com', '

# Numero de puntos en el domain


In [51]:
from codes.num_point_domain import num_point_domain
legit = dataset[dataset["status"] == "legitimate"]
df = legit["url"].tolist()
point = num_point_domain(df)
point.print_num()

['www.crestonwood.com', 'rgipt.ac.in', 'www.iracing.com', 'www.mutuo.it', 'vamoaestudiarmedicina.blogspot.com', 'parade.com', 'www.astrologyonline.eu', 'www.lifewire.com', 'technofizi.net', 'www.missfiga.com', 'www.chiefarchitect.com', 'www.2345daohang.com', 'www.game.co.uk', 'blog.hubspot.com', 'sophie-world.com', 'www.provenancevineyards.com', 'www.online-tech-tips.com', 'www.enkiquotes.com', 'answers.yahoo.com', 'www.chiostrodelbramante.it', 'yummy-cummy-in-my-tummy.tumblr.com', 'www.j-net.cn', 'press-preview.weebly.com', 'taccs.hu', 'wiki.openstreetmap.org', 'www.merriam-webster.com', 'www.bls.gov', 'ru.wikipedia.org', 'searchnetworking.techtarget.com', 'www.provenancevineyards.com', 'support.brother.com', 'www.excel-easy.com', 'stattrek.com', 'www.flaticon.com', 'www.bobsplans.com', 'en.wikipedia.org', 'www.history.com', 'remontka.pro', 'en.wikipedia.org', 'twitter.com', 'www.calculate.co.il', 'www.whio.com', 'www.markspcsolution.com', 'www.mummyandmini.com', 'www.showplaceicon.co

In [52]:
from codes.num_point_domain import num_point_domain
phishing = dataset[dataset["status"] == "phishing"]
df = phishing["url"].tolist()
point = num_point_domain(df)
point.print_num()

['shadetreetechnology.com', 'support-appleld.com.secureupdate.duilawyeryork.com', 'appleid.apple.com-app.es', 'www.shadetreetechnology.com', 'html.house', 'wave.progressfilm.co.uk', 'beta.kenaidanceta.com', 'www.ktplasmachinery.com', 'batvrms.net', 'support-appleld.com.secureupdate.duilawyeryork.com', 'polarklimatsgserver.blogspot.com', 'secureupdate.appleld.com.duilawyeryork.com', 'www.inbioma.pe', 'mno.naomigoff.com', 'xh167894743.el.r.appspot.com', 'dichvuvnpt.com', 'swallowthisbitchpics.com', 'realestateinvestordatabase.com', 'vufd.hyperphp.com', 'www.courgeon-immobilier.fr', 'grassform-my.sharepoint.com', 'hotmailsecure.brilliantbokeh.com', 'vaportotal.net', 'www.123formbuilder.com', 'sura.careervidi.com', 'phan.thuchao.free.fr', 'www.astroresidential.co.uk', 'www.nakamistrad.com', 'www.shadetreetechnology.com', 'sites.google.com', 'docs.google.com', 'sharedpont77wwjxjx.df.r.appspot.com', 'mailserver67187.uc.r.appspot.com', 'www.storybookalpacas.com', 'storage.cloud.google.com', '

# Numero de slash en la URL

In [53]:
from codes.num_slash_url import num_slash_url
legit = dataset[dataset["status"] == "legitimate"]
df = legit["url"].tolist()
point = num_slash_url(df)
point.print_num()


Estadísticas generales:
Mínimo: 2 barras
Máximo: 17 barras
Promedio: 3.83 barras


In [54]:
from codes.num_slash_url import num_slash_url
phishing = dataset[dataset["status"] == "phishing"]
df = phishing["url"].tolist()
point = num_slash_url(df)
point.print_num()


Estadísticas generales:
Mínimo: 2 barras
Máximo: 33 barras
Promedio: 4.75 barras


# Numero de puntos en la URL

In [55]:
from codes.num_point_url import num_point_url
legit = dataset[dataset["status"] == "legitimate"]
df = legit["url"].tolist()
point = num_point_url(df)
point.print_num()


Estadísticas generales:
Mínimo: 1 puntos
Máximo: 7 puntos
Promedio: 2.20 puntos


In [56]:
from codes.num_point_url import num_point_url
phishing = dataset[dataset["status"] == "phishing"]
df = phishing["url"].tolist()
point = num_point_url(df)
point.print_num()


Estadísticas generales:
Mínimo: 1 puntos
Máximo: 24 puntos
Promedio: 2.76 puntos


# Numero de dash en la URL

In [57]:
from codes.num_dash_url import num_dash_url
legit = dataset[dataset["status"] == "legitimate"]
df = legit["url"].tolist()
point = num_dash_url(df)
point.print_num()


Estadísticas generales:
Mínimo: 0 dash
Máximo: 43 dash
Promedio: 1.21 dash


In [58]:
from codes.num_dash_url import num_dash_url
phishing = dataset[dataset["status"] == "phishing"]
df = phishing["url"].tolist()
point = num_dash_url(df)
point.print_num()


Estadísticas generales:
Mínimo: 0 dash
Máximo: 32 dash
Promedio: 0.79 dash


# Preprocesamineto de datos

In [59]:
dataset

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate
...,...,...
11425,http://www.fontspace.com/category/blackletter,legitimate
11426,http://www.budgetbots.com/server.php/Server%20...,phishing
11427,https://www.facebook.com/Interactive-Televisio...,legitimate
11428,http://www.mypublicdomainpictures.com/,legitimate


In [60]:
from urllib.parse import urlparse

In [61]:
# Cambiar los valores de la columna 'status' a 0 para 'legitimate' y 1 para 'phishing'
dataset['status'] = dataset['status'].apply(lambda x: 0 if x == 'legitimate' else 1)

In [62]:
# Función para obtener el pathname y parameter de una URL
def extract_path_and_param(url):
    parsed_url = urlparse(url)
    path_and_param = parsed_url.path + parsed_url.params + parsed_url.query
    return path_and_param

# Aplicar la función a la columna 'url' para obtener el pathname y parameter
dataset['url'] = dataset['url'].apply(extract_path_and_param)

# Eliminar filas donde no hay pathname y parameter
dataset = dataset[dataset['url'] != '']
dataset = dataset[dataset['url'] != '/']

In [63]:
dataset

,url,status
0,/router.php,0
1,/V4/validation/a111aedc8ae390eabcfa130e041a10a4,1
2,/ap/89e6a3b4b063b8d/cmd=_update&dispatch=89e6a...,1
4,/tracks/gateway-motorsports-park/,0
7,/V4/validation/ba4b8bddd7958ecb8772c836c2969531,1
...,...,...
11424,/Q/What_are_the_sizes_of_computer_memory,0
11425,/category/blackletter,0
11426,/server.php/Server%20update/index.phpemail=USE...,1
11427,/Interactive-Television-Pvt-Ltd-Group-M-100230...,0


# Visualizacion de resultados

In [64]:
import pandas_profiling

# Cargar tu dataset
# Asegúrate de que la variable 'dataset' esté correctamente definida antes de ejecutar este código
# dataset = ...

# Crear el perfil con pandas_profiling
profile = pandas_profiling.ProfileReport(dataset, config_file=None)

# Generar el informe y guardarlo como un archivo HTML
profile.to_file("informe_perfil.html")


TypeCheckError: argument "config_file" (None) did not match any element in the union:
  pathlib.Path: is not an instance of pathlib.Path
  str: is not an instance of str